In [1]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook
import numpy as np
import pandas as pd


In [2]:
b_test = pd.read_csv('models/shadow_cifar10_1000_badnet/b_train.csv',header=None)
c_test = pd.read_csv('../../test/Reprogramming/resnet18/shadow_cifar10_1000_none/c_train.csv',header=None)

In [3]:
c_test = c_test.transpose() 
b_test = b_test.transpose() 

In [4]:
c_test = c_test.values
b_test = b_test.values

In [5]:
c_test.shape, b_test.shape

((11, 240000), (11, 152000))

In [6]:
96000 / 8000

12.0

In [7]:
test_backdoor = b_test[:-1, :]
test_clean = c_test[:-1, :152000]

In [8]:
test_backdoor.shape, test_clean.shape

((10, 152000), (10, 152000))

In [9]:
96000 / 24

4000.0

In [10]:
sub_matrices = np.split(test_clean, 19, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
test_c = np.concatenate(flat_matrices)

print(test_c.shape)  # 输出 (15, 40000)

(19, 80000)


In [11]:
sub_matrices = np.split(test_backdoor, 19, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
test_b = np.concatenate(flat_matrices)

print(test_b.shape)  # 输出 (15, 40000)

(19, 80000)


In [12]:
res_clean  = np.concatenate((test_c,  np.ones((len(test_c[: ,-1]), 1))), axis=1)

In [13]:
res_backdoor  = np.concatenate((test_b,  np.zeros((len(test_b[: ,-1]), 1))), axis=1)

In [14]:
res_clean.shape, res_backdoor.shape

((19, 80001), (19, 80001))

In [15]:
model_results = np.vstack((res_clean, res_backdoor))

In [16]:
model_results.shape

(38, 80001)

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
model_label = model_results[:, -1]

In [19]:
model = model_results[:, :-1]

In [20]:
forest = RandomForestClassifier(n_estimators = 10000)

In [21]:
forest_fit = forest.fit(model,model_label)

In [40]:
b = pd.read_csv('models/target_cifar10_50000_badnet/b_train.csv',header=None)
c = pd.read_csv('models/target_none_cifar10_50000_blend/c_train.csv',header=None)

In [41]:
c = c.transpose() 
b = b.transpose() 

In [42]:
c.shape, b.shape

((11, 120000), (11, 120000))

In [43]:
c = c.values 
b = b.values

In [44]:
test_backdoor = b[:-1, :]
test_clean = c[:-1, :]

In [45]:
test_backdoor.shape

(10, 120000)

In [46]:
sub_matrices = np.split(test_clean, 15, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
c = np.concatenate(flat_matrices)

print(c.shape)  # 输出 (15, 40000)

(15, 80000)


In [47]:
sub_matrices = np.split(test_backdoor, 15, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
b = np.concatenate(flat_matrices)

print(b.shape)  # 输出 (15, 40000)

(15, 80000)


In [48]:
mem_res_clean = forest.predict(c)
mem_res_backdoor = forest.predict(b)

In [49]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

In [50]:
y_pred = np.hstack((mem_res_clean, mem_res_backdoor))
y_true = np.concatenate((np.ones_like(mem_res_clean), np.zeros_like(mem_res_backdoor)))

In [51]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

In [52]:
recall_score(y_true, y_pred)

1.0

In [53]:
print(fp/(fp+tn))

0.0


In [54]:
f1_score(y_true, y_pred)

1.0

In [55]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred)

In [56]:
roc_auc = auc(fpr, tpr)

In [57]:
plt.show()
print(roc_auc)

1.0
